In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
from collections import deque


# Excepciones personalizadas


class UsuarioExistenteError(Exception):
    pass


class UsuarioNoExisteError(Exception):
    pass


class AmistadExistenteError(Exception):
    pass


class AmistadInvalidaError(Exception):
    pass


# Modelo de usuario


class Usuario:
    def __init__(self, nombre: str):
        self.nombre = nombre
        self.amigos = set()

    def agregar_amigo(self, amigo):
        if amigo == self:
            raise AmistadInvalidaError(f"Un usuario no puede ser su propio amigo: {self.nombre}")
        if amigo in self.amigos:
            raise AmistadExistenteError(f"{amigo.nombre} ya es amigo de {self.nombre}")
        self.amigos.add(amigo)

    def __repr__(self):
        return f"Usuario({self.nombre})"


# Modelo de red social


class RedSocial:
    def __init__(self):
        self.usuarios = {}

    def agregar_usuario(self, nombre: str):
        if nombre in self.usuarios:
            raise UsuarioExistenteError(f"El usuario {nombre} ya existe.")
        self.usuarios[nombre] = Usuario(nombre)

    def agregar_amigo(self, nombre1: str, nombre2: str):
        if nombre1 not in self.usuarios or nombre2 not in self.usuarios:
            raise UsuarioNoExisteError("Uno o ambos usuarios no existen.")
        usuario1 = self.usuarios[nombre1]
        usuario2 = self.usuarios[nombre2]
        usuario1.agregar_amigo(usuario2)
        usuario2.agregar_amigo(usuario1)

    def recomendar_amigos_bfs(self, nombre: str):
        if nombre not in self.usuarios:
            raise UsuarioNoExisteError(f"El usuario {nombre} no existe.")
        usuario_inicio = self.usuarios[nombre]
        visitados = set()
        recomendaciones = set()
        cola = deque([(usuario_inicio, 0)])
        visitados.add(usuario_inicio)

        while cola:
            actual, nivel = cola.popleft()
            if nivel == 2:
                recomendaciones.add(actual)
            elif nivel < 2:
                for amigo in actual.amigos:
                    if amigo not in visitados:
                        visitados.add(amigo)
                        cola.append((amigo, nivel + 1))

        recomendaciones.discard(usuario_inicio)
        recomendaciones -= usuario_inicio.amigos
        return recomendaciones


# Funciones de optimización


def bfs_sin_numba(grafo_np, inicio):
    visitados = set([inicio])
    cola = deque([inicio])
    while cola:
        actual = cola.popleft()
        for vecino in np.where(grafo_np[actual] == 1)[0]:
            if vecino not in visitados:
                visitados.add(vecino)
                cola.append(vecino)
    return visitados


@njit
def bfs_con_numba(grafo_np, inicio):
    visitados = set([inicio])
    cola = [inicio]
    while cola:
        actual = cola.pop(0)
        for vecino in range(grafo_np.shape[0]):
            if grafo_np[actual, vecino] == 1 and vecino not in visitados:
                visitados.add(vecino)
                cola.append(vecino)
    return visitados


# Funciones de benchmark


def crear_grafo_lineal(n):
    grafo = np.zeros((n, n), dtype=np.int8)
    for i in range(n - 1):
        grafo[i, i + 1] = 1
        grafo[i + 1, i] = 1
    return grafo


def medir_tiempo(func, *args):
    t0 = time.time()
    func(*args)
    t1 = time.time()
    return t1 - t0


def visualizar_tiempos(tiempos, etiquetas):
    plt.bar(etiquetas, tiempos, color=["red", "green"])
    plt.ylabel("Tiempo (s)")
    plt.title("Comparación de tiempos BFS")
    plt.show()


# Demostración


def demo_red_social():
    red = RedSocial()
    usuarios_demo = ["Ana", "Luis", "Marta", "Carlos", "Sofía", "Pedro"]
    for nombre in usuarios_demo:
        try:
            red.agregar_usuario(nombre)
        except UsuarioExistenteError as e:
            print(e)

    amistades = [("Ana", "Luis"), ("Luis", "Marta"), ("Marta", "Carlos"), ("Carlos", "Sofía"), ("Sofía", "Pedro")]

    for u1, u2 in amistades:
        try:
            red.agregar_amigo(u1, u2)
        except (UsuarioNoExisteError, AmistadExistenteError, AmistadInvalidaError) as e:
            print(e)

    print("\nRecomendaciones para Ana:")
    recomendaciones = red.recomendar_amigos_bfs("Ana")
    for r in recomendaciones:
        print(f"- {r.nombre}")


def demo_benchmark():
    N = 1000
    grafo_np = crear_grafo_lineal(N)

    t_sin_numba = medir_tiempo(bfs_sin_numba, grafo_np, 0)
    t_con_numba = medir_tiempo(bfs_con_numba, grafo_np, 0)

    print(f"\nTiempo sin Numba: {t_sin_numba:.6f} segundos")
    print(f"Tiempo con Numba: {t_con_numba:.6f} segundos")
    print("El BFS tiene un rendimiento O(V + E), donde V son los nodos y E los enlaces.")

    visualizar_tiempos([t_sin_numba, t_con_numba], ["Sin Numba", "Con Numba"])


if __name__ == "__main__":
    demo_red_social()
    demo_benchmark()
